# 2.3 Concept Gradient 
use autograd
# 2.3.1 Concept 
.requires_grad - True
use .backward() to compute gradient 
pass in a tensor as y when use y.backward()
.detach() avoid trace
with torch.no_grad()

Every Tensor has attribyte of .grad_fn 

In [2]:
import torch

In [8]:
#可训练参数（requires_grad=True）的梯度。
#创建一个Tensor并设置requires_grad=True:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

#.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，
# 若是，则grad_fn返回一个与这些运算相关的对象，否则是None。
# x is a leaf node , grad_fn = NONE 

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [9]:
#create y to do some operation
y = x + 2
print(y)
print(y.grad_fn)
#而y是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [10]:
#more complex operation
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [4]:
a = torch.randn(2, 2,requires_grad=True)
print(a.requires_grad)



True


In [5]:
b = (a * a).sum()
print(b.grad_fn)


In [6]:
print(a)
print(b)

tensor([[ 2.0594,  1.1848],
        [ 0.0627, -0.0526]], requires_grad=True)
tensor(5.6516, grad_fn=<SumBackward0>)


# 2.3.3 Gradient

In [11]:
out

tensor(27., grad_fn=<MeanBackward0>)

In [12]:
out.backward() # 等价于 out.backward(torch.tensor(1.))


In [14]:
print(x)
print(x.grad)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [16]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
print(out2)

out2.backward()
print(x.grad)

tensor(4., grad_fn=<SumBackward0>)
tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])


In [18]:
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


In [20]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [21]:
#z 不是一个标量，所以在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量。
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)



tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [26]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True


True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [27]:
y3.backward()
print(x.grad)

tensor(2.)


In [28]:
#use tensor.data
x = torch.ones(1,requires_grad=True)

print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
